In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230323_sqli_colab'

In [14]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.8.*"

In [15]:
!pip install -q tf-models-official==2.7.0

In [1]:
from pathlib import Path
from datetime import datetime
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

import pandas as pd
import time


import time
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

print('Pandas v', pd.__version__)
#tf.get_logger().setLevel('ERROR')

ModuleNotFoundError: No module named 'official'

In [17]:
 !nvidia-smi

Tue Feb 14 19:36:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    53W / 400W |   9208MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
%%script echo skipping
#debug 
# main_folder = Path('/content/drive/MyDrive/Colab Notebooks/kasim/2023 Feb Bert SQLi/')
main_folder = Path('/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230323_sqli_colab')

results_file = main_folder / 'results_230202_130230.pkl'
results_file_bert = main_folder / 'results_230214_135946.pkl'
print(results_file.is_file())
print(results_file_bert.is_file())

df = pd.read_pickle(results_file_bert)
df.to_pickle('results_230214_135946_protocol4.pkl', protocol=4)

skipping


In [ ]:
!python main.py

In [19]:
import pandas as pd
import os
from datetime import datetime

from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score

class Recorder:
  def __init__(self, file_name):
    print("Recorder")
    self.method_name = ""
    self.file_name = file_name
    self.dictResults = {}
    self.dfResults = pd.DataFrame({'A' : []})

  def set_current_method(self, method_name):
    """Change the name of the current method. The table row index will be this value.

    Args:
        method_name (string): the name of the method e.g. "bow nb"
    """    
    self.method_name = method_name
    print(f"current method name: {self.method_name}")
    self.dictResults.update( {self.method_name : {} } )

  def add_or_update_field(self, field, value):
    self.dictResults[self.method_name].update( {field : value} )
  
  def to_dataFrame(self):
    print("package.record.to_dataFrame()")
    self.dfResults = pd.DataFrame.from_dict(self.dictResults, orient='index')
    return self.dfResults

  def df_to_pickle(self):
    print("package.record.df_to_pickle()")
    
    currentDateAndTime = datetime.now()
    cwd = os.getcwd()
    currentTime = currentDateAndTime.strftime("%y%m%d_%H%M%S")
    output_filename = self.file_name # f"results_{currentTime}.pkl"
    print(f"Saving results dataframe as pandas pickle to : {output_filename}" ) 
    self.to_dataFrame()
    self.dfResults.to_pickle(output_filename, protocol=4)
    print("Pickle Saved.")

  def metrics(self, y_true, y_pred):
    precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average='binary')
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    self.add_or_update_field("overall_accuracy", acc)
    self.add_or_update_field("precision", precision)
    self.add_or_update_field("recall", recall)
    self.add_or_update_field("f1", f1)
    #package.record.add_or_update_field("support", support)
    self.add_or_update_field("tn", tn)
    self.add_or_update_field("fp", fp)
    self.add_or_update_field("fn", fn)
    self.add_or_update_field("tp", tp)

In [20]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [21]:
# Manual selection
bert_model_name_list = ['bert_en_uncased_L-12_H-768_A-12', 
                        'bert_en_cased_L-12_H-768_A-12',
                        'small_bert/bert_en_uncased_L-2_H-128_A-2',
                        'small_bert/bert_en_uncased_L-4_H-512_A-8',
                        'small_bert/bert_en_uncased_L-8_H-128_A-2',
                        'small_bert/bert_en_uncased_L-12_H-768_A-12',
                        'bert_multi_cased_L-12_H-768_A-12',
                       'albert_en_base',
                        'electra_base',
                        'electra_small']

In [22]:
def build_classifier_model(local_tfhub_handle_preprocess, local_tfhub_handle_encoder):
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Payload')
  preprocessing_layer = hub.KerasLayer(local_tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(local_tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [23]:
main_dir = Path('/content/drive/MyDrive/Colab Notebooks/kasim/2023 Feb Bert SQLi/')
# 'SQLiV3_train.tsv', 'SQLiV3_test.tsv'
# 'NEW_normal_plus_waf_bypass_dataset_testing.tsv'
# 'NEW_normal_plus_waf_bypass_dataset_training.tsv'
#train_file_name = 'NEW_normal_plus_waf_bypass_dataset_training.tsv'
train_file_name = 'SQLiV3_train.tsv'
test_file_name = 'SQLiV3_test.tsv'
train_file = Path(main_dir / train_file_name)
test_file = Path(main_dir / test_file_name)

print(f'Train file exists: {train_file.is_file()}')
print(f'Test file exists: {test_file.is_file()}')

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%y%m%d_%H%M%S")
recording = Recorder(main_dir / f'results_{currentTime}.pkl')

	# Load Training dataset
data_train = pd.read_csv(train_file, sep='\t', engine='python')
train_size = data_train.shape[0]

	# Load Testing dataset
data_test = pd.read_csv(test_file, sep='\t', engine='python')
test_size = data_test.shape[0]
train_labels = data_train.pop('Attacker')
data_train_dict = data_train.to_dict()


train_csv_ds = tf.data.experimental.make_csv_dataset(
    str(train_file),
    field_delim='\t',
    use_quote_delim=False,
    batch_size=16, 
    label_name='Attacker',
    num_epochs=1,
    ignore_errors=True,)

test_csv_ds = tf.data.experimental.make_csv_dataset(
    str(test_file),
    field_delim='\t',
    use_quote_delim=False,
    batch_size=16, 
    label_name='Attacker',
    num_epochs=1,
    ignore_errors=True,)

AUTOTUNE = tf.data.AUTOTUNE
batch_size = 16
seed = 42

train_ds = test_csv_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = test_csv_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = val_ds

class_names = ['Legit','Attacker']



Train file exists: True
Test file exists: True
Recorder


In [24]:


#train_ds = train_ds.take(8)
#val_ds = val_ds.take(8)
#test_ds = test_ds.take(8)

for bert_model_name in bert_model_name_list:
  #bert_model_name = bert_model_name_list[0]
  print('****************************************************')
  recording.set_current_method(f"{bert_model_name}" )
  tfhub_handle_encoder = map_name_to_handle[bert_model_name]
  tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

  print(f'BERT model selected           : {tfhub_handle_encoder}')
  print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')


  bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
  bert_model = hub.KerasLayer(tfhub_handle_encoder)
  classifier_model = build_classifier_model(tfhub_handle_preprocess, tfhub_handle_encoder)

  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  metrics = tf.metrics.BinaryAccuracy()

  epochs = 10
  #steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
  steps_per_epoch = data_train.size
  num_train_steps = steps_per_epoch * epochs
  num_warmup_steps = int(0.1*num_train_steps)

  init_lr = 3e-5
  optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')

  classifier_model.compile(optimizer=optimizer,
                          loss=loss,
                          metrics=metrics)


  print(f'Training model with {tfhub_handle_encoder}')
  start_time = time.time()
  history = classifier_model.fit(x=train_ds,
                                validation_data=val_ds,
                                epochs=epochs)
  stop_time = time.time()
  training_time = stop_time - start_time

  loss, accuracy = classifier_model.evaluate(test_ds)

  print(f'Loss: {loss}')
  print(f'Accuracy: {accuracy}')

  start_time = time.time()
  y_pred = classifier_model.predict(test_ds)
  stop_time = time.time()
  testing_time = stop_time - start_time
  y_pred_np = tf.cast(tf.sigmoid(y_pred) > 0.5, tf.int32).numpy()
  X, y_true = zip(*test_ds.unbatch())
  y_true_np = [y.numpy() for y in y_true]
  #precision, recall, f1, support = precision_recall_fscore_support(y_true_np, y_pred_np, average='binary')
  #tn, fp, fn, tp = confusion_matrix(y_true_np, y_pred_np).ravel()

  #print(precision, recall, f1, support)
  #print(tn, fp, fn, tp )
  #print(training_time, testing_time)

  #recording.add_or_update_field("tn", tn)
  #recording.add_or_update_field("fp", fp)
  #recording.add_or_update_field("fn", fn)
  #recording.add_or_update_field("tp", tp)

  #recording.add_or_update_field("precision", precision)
  #recording.add_or_update_field("recall", recall)
  #recording.add_or_update_field("f1", f1)
  recording.metrics(y_true_np, y_pred_np)

  recording.add_or_update_field(field="training time", value=training_time)
  recording.add_or_update_field(field="testing time", value=testing_time)
  recording.add_or_update_field(field="train size", value=train_size)
  recording.add_or_update_field(field="test size", value=test_size)
  recording.add_or_update_field(field="train data", value=train_file_name)
  recording.add_or_update_field(field="test data", value=test_file_name)


  print(recording.to_dataFrame())

****************************************************
current method name: bert_en_uncased_L-12_H-768_A-12
BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/10
383/383 [==============================] - 60s 114ms/step - loss: 0.4201 - binary_accuracy: 0.8244 - val_loss: 0.0466 - val_binary_accuracy: 0.9910
Epoch 2/10
383/383 [==============================] - 42s 110ms/step - loss: 0.0334 - binary_accuracy: 0.9923 - val_loss: 0.0175 - val_binary_accuracy: 0.9966
Epoch 3/10
383/383 [==============================] - 42s 111ms/step - loss: 0.0201 - binary_accuracy: 0.9962 - val_loss: 0.0069 - val_binary_accuracy: 0.9980
Epoch 4/10
383/383 [==============================] - 42s 111ms/step - loss: 0.0127 - binary_accuracy: 0.9975 - val_loss: 0.0023 - val_binary_accurac

In [25]:
recording.df_to_pickle() 

package.record.df_to_pickle()
Saving results dataframe as pandas pickle to : /content/drive/MyDrive/Colab Notebooks/kasim/2023 Feb Bert SQLi/results_230214_193603.pkl
package.record.to_dataFrame()
Pickle Saved.
